In [1]:
# Import the necessary libraries

import glob
import time
import duckdb
import pandas as pd

In [2]:
# Establish a connection to duckdb

conn = duckdb.connect()

In [3]:
# Read the contents of the csv file using duckdb

cur_time = time.time()
df = conn.execute("""
    SELECT *
    FROM read_csv_auto("./data/Sales_Product_Combined.csv", header=True)
    """).df()
print(f"time: ({time.time() - cur_time})")
print(df)

time: (0.40958714485168457)
        Order ID                     Product  Quantity Ordered   Price  \
0         176558        USB-C Charging Cable                 2   11.95   
1         176559  Bose SoundSport Headphones                 1   99.99   
2         176560                Google Phone                 1     600   
3         176560            Wired Headphones                 1   11.99   
4         176561            Wired Headphones                 1   11.99   
...          ...                         ...               ...     ...   
185945    259353      AAA Batteries (4-pack)                 3    2.99   
185946    259354                      iPhone                 1     700   
185947    259355                      iPhone                 1     700   
185948    259356      34in Ultrawide Monitor                 1  379.99   
185949    259357        USB-C Charging Cable                 1   11.95   

       Order Date      Time                         Purchase Address  \
0      2019

In [4]:
# Create a view

conn.register("df_view", df)

In [5]:
# Check the schema of the view

conn.execute("DESCRIBE df_view").df()

,column_name,column_type,null,key,default,extra
0,Order ID,BIGINT,YES,None,None,NaN
1,Product,VARCHAR,YES,None,None,NaN
2,Quantity Ordered,BIGINT,YES,None,None,NaN
3,Price,VARCHAR,YES,None,None,NaN
4,Order Date,TIMESTAMP_NS,YES,None,None,NaN
5,Time,VARCHAR,YES,None,None,NaN
6,Purchase Address,VARCHAR,YES,None,None,NaN
7,City,VARCHAR,YES,None,None,NaN
8,Product Type,VARCHAR,YES,None,None,NaN


In [6]:
# Count the total number of records

conn.execute("SELECT count(*) FROM df_view").df()

,count_star()
0,185950


In [7]:
# Count the total number of null value and remove them

df.isnull().sum()
df = df.dropna(how='all')

In [8]:
# A test implementation of a where clause

conn.execute("""SELECT * FROM df WHERE "Order ID"='295665'""").df()

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,295665,Macbook Pro Laptop,1,"1,700.00",2019-12-30,12:01 AM,"136 Church St, New York City, NY 10001",New York City,Laptop


In [9]:
# Create a table from the dataframe and enforcing the appropriate schema

conn.execute("""
             CREATE OR REPLACE TABLE sales AS
                SELECT
                    "Order ID"::INTEGER AS order_id,
                    Product AS product,
                    "Quantity Ordered"::INTEGER AS quantity,
                    REPLACE("Price", ',', '')::DECIMAL AS price,
                    strptime("Order Date", '%Y-%m-%d %H:%M:%S')::DATE as order_date,
                    "Purchase Address" AS purchase_address
                FROM df
                WHERE
                    TRY_CAST("Order ID" AS INTEGER) NOTNULL    
             """)

In [11]:
# Excluding records from a select statement

conn.execute(
    "SELECT * EXCLUDE (product, order_date, purchase_address) FROM sales").df()

,order_id,quantity,price
0,176558,2,11.95
1,176559,1,99.99
2,176560,1,600.00
3,176560,1,11.99
4,176561,1,11.99
...,...,...,...
185945,259353,3,2.99
185946,259354,1,700.00
185947,259355,1,700.00
185948,259356,1,379.99


In [12]:
# Minimum value implementation across selected columns

conn.execute("""
             SELECT 
                MIN(COLUMNS(* EXCLUDE (product, order_date, purchase_address))) FROM sales
             """).df()

,min(sales.order_id),min(sales.quantity),min(sales.price)
0,141234,1,2.99


In [13]:
# Creating a view from a table

conn.execute("""
             CREATE OR REPLACE VIEW aggregated_sales AS
             SELECT 
                order_id,
                COUNT (1) AS nb_orders,
                MONTH(order_date) AS month,
                STR_SPLIT(purchase_address, ',')[2] AS city,
                SUM(quantity * price) AS revenue
            FROM sales
            GROUP BY ALL
             """)

In [14]:
conn.execute("FROM aggregated_sales").df()

,order_id,nb_orders,month,city,revenue
0,196940,1,5,New York City,149.99
1,196941,1,5,Boston,7.68
2,196945,1,5,Austin,600.00
3,196947,1,5,New York City,11.95
4,196949,1,5,San Francisco,5.98
...,...,...,...,...,...
178432,173740,2,3,San Francisco,249.99
178433,175808,1,3,Atlanta,11.95
178434,196623,1,5,Dallas,150.00
178435,173744,1,3,Portland,150.00


In [15]:
# Total revenue across the cities

conn.execute("""
             SELECT
                city,
                SUM(revenue) AS total
            FROM aggregated_sales
            GROUP BY city
            ORDER BY total DESC
             """).df()

,city,total
0,San Francisco,8262203.91
1,Los Angeles,5452570.80
2,New York City,4664317.43
3,Boston,3661642.01
4,Atlanta,2795498.58
5,Dallas,2767975.40
6,Seattle,2747755.48
7,Portland,2320490.61
8,Austin,1819581.75


In [16]:
# Saving as a parquet file

conn.execute(
    "COPY (FROM aggregated_sales) TO 'aggregated_sales.parquet' (FORMAT 'parquet')")

In [17]:
# Reading a parquet file

conn.execute("FROM aggregated_sales.parquet").df()

,order_id,nb_orders,month,city,revenue
0,196944,1,5,Los Angeles,149.99
1,196950,1,5,New York City,11.99
2,196951,1,5,San Francisco,150.00
3,196975,1,5,Portland,150.00
4,196977,1,5,New York City,11.95
...,...,...,...,...,...
178432,175267,2,3,Atlanta,711.99
178433,194289,1,5,Dallas,700.00
178434,195762,1,5,San Francisco,1700.00
178435,196739,1,5,San Francisco,400.00
